In [ ]:
!pip install Flask-RESTful

In [ ]:
!pip install Flask

In [ ]:
import torch
import clip
import cv2
import numpy as np
from PIL import Image
from flask import Flask
from flask_restful import Resource, Api, reqparse
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [3]:
THRESHOLD = 0.27
NUM_FRAMES = 5

In [15]:
def extract_frames(video_path):
    vidcap = cv2.VideoCapture(video_path)
    frames = []
    success,image = vidcap.read()
    count = 0
    while success:
        frames.append(Image.fromarray(image))
        success,image = vidcap.read()
        count += 1
    return frames

In [4]:
def check(score):
    curr_score = 0
    i = 0
    cnt = 0
    while i < NUM_FRAMES and i < len(score):
        curr_score += score[i]
        i += 1
    if(curr_score/NUM_FRAMES >= THRESHOLD):
        cnt+=1
    while i < len(score):
        curr_score += score[i];
        curr_score -= score[i- NUM_FRAMES]
        if(curr_score/NUM_FRAMES >= THRESHOLD):
            cnt+=1
        i+=1
    return cnt

In [5]:
def process_videos(video_paths):
    res = []
    for video_path in video_paths:
        frames = extract_frames(video_path)
        curr_vid = []
        for img in frames:
            image = preprocess(img).unsqueeze(0).to(device)
            with torch.no_grad():
                image = model.encode_image(image).float()
                image /= image.norm(dim=-1, keepdim=True)
            curr_vid.append(image)
        res.append(curr_vid)
    return res

In [44]:
def search_preprocessed_data(data, text):
    res = []
    text = clip.tokenize(text).to(device)
    with torch.no_grad():
        text = model.encode_text(text).float()
    text /= text.norm(dim=-1, keepdim=True)
    i = 0
    all_score = []
    for vid in data:
        score = []
        for frame in vid:
            similarity = text.cpu().numpy() @ frame.cpu().numpy().T
            score.append(similarity)
        if(check(score)):
            res.append(i)
        i += 1
        all_score.append(score)
    return res, all_score

In [45]:
app = Flask(__name__)
api = Api(app)

In [26]:
path = os.getcwd()
path = path + '\dataset'
video_paths = [path + '\\bird.mp4',path + '\\cat.mp4', path + '\\fish.mp4', path + '\\flower.mp4' ]
data = process_videos(video_paths)

In [27]:
links = ["https://pixabay.com/videos/budgerigar-bird-parrot-pet-animal-2471/","https://pixabay.com/videos/cute-cat-funny-cat-kitten-domestic-3092/",
"https://pixabay.com/videos/fishes-small-aquarium-small-fish-16166/", "https://pixabay.com/videos/blue-tit-blossom-tree-blue-tit-3063/"]
d = {}
i = 0
for vid in links:
    d[i] = vid
    i+=1

In [46]:
class clipSearch(Resource):
    def get(self):
        parser = reqparse.RequestParser()
        parser.add_argument('text', required=True)
        args = parser.parse_args()
        text = args['text']
        ans = []
        res, scores = search_preprocessed_data(data, text)
        for vid in res:
            ans.append({'link': d[vid], 'path': video_paths[vid]})
        return {'data': ans}, 
api.add_resource(clipSearch, '/clipsearch') 

In [47]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


0


127.0.0.1 - - [31/Mar/2023 18:41:21] "GET /clipsearch?text=fish HTTP/1.1" 200 -


[[array([[0.21342525]], dtype=float32), array([[0.21797691]], dtype=float32), array([[0.2215333]], dtype=float32), array([[0.22311033]], dtype=float32), array([[0.22201009]], dtype=float32), array([[0.219716]], dtype=float32), array([[0.22059838]], dtype=float32), array([[0.22366478]], dtype=float32), array([[0.22653358]], dtype=float32), array([[0.22839111]], dtype=float32), array([[0.22268349]], dtype=float32), array([[0.21925515]], dtype=float32), array([[0.22108084]], dtype=float32), array([[0.22089088]], dtype=float32), array([[0.21809119]], dtype=float32), array([[0.21815102]], dtype=float32), array([[0.21577]], dtype=float32), array([[0.21806023]], dtype=float32), array([[0.2179888]], dtype=float32), array([[0.21800925]], dtype=float32), array([[0.21278873]], dtype=float32), array([[0.2112799]], dtype=float32), array([[0.2102548]], dtype=float32), array([[0.21085005]], dtype=float32), array([[0.2168227]], dtype=float32), array([[0.2194004]], dtype=float32), array([[0.21510263]],

...

[[array([[0.19244751]], dtype=float32), array([[0.19742985]], dtype=float32), array([[0.20257907]], dtype=float32), array([[0.2021382]], dtype=float32), array([[0.20243895]], dtype=float32), array([[0.20175463]], dtype=float32), array([[0.20365006]], dtype=float32), array([[0.20520853]], dtype=float32), array([[0.20742604]], dtype=float32), array([[0.2082457]], dtype=float32), array([[0.20408165]], dtype=float32), array([[0.20180802]], dtype=float32), array([[0.20487899]], dtype=float32), array([[0.20494312]], dtype=float32), array([[0.2025407]], dtype=float32), array([[0.20410307]], dtype=float32), array([[0.20165755]], dtype=float32), array([[0.20355892]], dtype=float32), array([[0.20338224]], dtype=float32), array([[0.20113748]], dtype=float32), array([[0.19703044]], dtype=float32), array([[0.1957949]], dtype=float32), array([[0.19503061]], dtype=float32), array([[0.19510952]], dtype=float32), array([[0.19905998]], dtype=float32), array([[0.20398492]], dtype=float32), array([[0.2005

127.0.0.1 - - [31/Mar/2023 18:50:46] "GET /clipsearch?text=apple HTTP/1.1" 200 -


[[array([[0.19773123]], dtype=float32), array([[0.2012674]], dtype=float32), array([[0.20327882]], dtype=float32), array([[0.20532179]], dtype=float32), array([[0.2057589]], dtype=float32), array([[0.20626466]], dtype=float32), array([[0.20605835]], dtype=float32), array([[0.20671211]], dtype=float32), array([[0.20746677]], dtype=float32), array([[0.20775312]], dtype=float32), array([[0.20418325]], dtype=float32), array([[0.20293733]], dtype=float32), array([[0.20546158]], dtype=float32), array([[0.2059575]], dtype=float32), array([[0.20597163]], dtype=float32), array([[0.20669666]], dtype=float32), array([[0.20452997]], dtype=float32), array([[0.20670687]], dtype=float32), array([[0.20640345]], dtype=float32), array([[0.20635727]], dtype=float32), array([[0.20175095]], dtype=float32), array([[0.202943]], dtype=float32), array([[0.20274489]], dtype=float32), array([[0.20389956]], dtype=float32), array([[0.20913947]], dtype=float32), array([[0.21055558]], dtype=float32), array([[0.20579

127.0.0.1 - - [31/Mar/2023 18:50:53] "GET /clipsearch?text=ball HTTP/1.1" 200 -


[[array([[0.2273448]], dtype=float32), array([[0.23217018]], dtype=float32), array([[0.23542114]], dtype=float32), array([[0.23553152]], dtype=float32), array([[0.23396567]], dtype=float32), array([[0.23189515]], dtype=float32), array([[0.23318446]], dtype=float32), array([[0.2348391]], dtype=float32), array([[0.23534457]], dtype=float32), array([[0.23350324]], dtype=float32), array([[0.22961636]], dtype=float32), array([[0.22618522]], dtype=float32), array([[0.22850603]], dtype=float32), array([[0.22884224]], dtype=float32), array([[0.22673836]], dtype=float32), array([[0.22788873]], dtype=float32), array([[0.22715561]], dtype=float32), array([[0.23196596]], dtype=float32), array([[0.23156519]], dtype=float32), array([[0.2303487]], dtype=float32), array([[0.22672696]], dtype=float32), array([[0.22668366]], dtype=float32), array([[0.22713605]], dtype=float32), array([[0.23038189]], dtype=float32), array([[0.2369391]], dtype=float32), array([[0.2390515]], dtype=float32), array([[0.23308